In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# from model import Create_transfert_learning_Model
from processing.generator import Dataset_Generator
from processing.processor import Image_Processing

# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [6]:
print(tf.config.list_physical_devices('GPU'))
print(tf.config.experimental.get_memory_usage('GPU:0'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
0


In [7]:
Generator = Dataset_Generator(nbr_augmentation=1)

In [8]:
X_train, Y_train,\
X_dev, Y_dev,\
X_test, Y_test = Generator(healthy_dir_name='leaves_img/sain',
                           disease_dir_name='leaves_img/malade')

100%|██████████| 118/118 [00:01<00:00, 70.30it/s]


(1475, 224, 224, 3) (1475, 1)
(56, 224, 224, 3) (56, 1)
(114, 224, 224, 3) (114, 1)


In [9]:
print(sys.getsizeof(X_train))
print(sys.getsizeof(Y_train))
print(sys.getsizeof(X_dev))
print(sys.getsizeof(Y_dev))
print(sys.getsizeof(X_test))
print(sys.getsizeof(Y_test))

888115360
128
33718432
128
68640928
128


In [17]:
print(tf.config.experimental.get_memory_usage('GPU:0')* 1e-9, "GB")

0.9743866880000001 GB


In [18]:
from tensorflow.keras.optimizers import Adam
from keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten 

In [19]:
def load_vgg(IMAGE_SHAPE : list):

  vgg = tf.keras.applications.VGG19(include_top=False, 
                                    input_shape=IMAGE_SHAPE,
                                    weights='imagenet')
  for layer in vgg.layers:
    layer.trainable = False

  return (vgg)

In [20]:
def add_new_top_layers(vgg : Model):
    
    last_conv_layer = vgg.output
    flatten_layer = Flatten(name='flatten_layer')(last_conv_layer)
    fully_connected = Dense(4096, name='fully_connected')(flatten_layer) 
    prediction = Dense(2, activation='sigmoid', name='lamioude')(fully_connected)
    
    model = Model(inputs=vgg.input, outputs=prediction)
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    return (model)

In [21]:
from tensorflow.keras import Sequential

In [22]:
class Mildew_Classifier(Model):

    def __init__(self, IMAGE_SHAPE=[224, 224, 3]):
      
      super().__init__()
      self.vgg = load_vgg(IMAGE_SHAPE)
      self.flatten = Flatten(name='Flatten')
      self.FC = Dense(4096, name='FC')
      self.prediction = Dense(2, activation='softmax', name='prediction')
      self.compile(optimizer="adam",
                   loss="mse",
                   metrics=["acuracy"])
  
    def call(self, input):
      
      x = self.vgg(input)
      x = self.flatten(x)
      x = self.FC(x)
      output = self.prediction(x)
  
      return (output)

In [23]:
model = Mildew_Classifier()

In [24]:
print(tf.config.experimental.get_memory_usage('GPU:0')* 1e-9, "GB")

1.0555028480000002 GB


In [13]:
history = model.fit(X_train, Y_train, epochs=1, batch_size=1, verbose=1, shuffle=True)

ValueError: in user code:

    File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/keras/engine/training.py", line 817, in train_step
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/keras/engine/compile_utils.py", line 439, in update_state
        self.build(y_pred, y_true)
    File "/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/keras/engine/

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 10))

ax[0].plot(history.history['loss'])
ax[0].set_title("Loss after epochs")
ax[0].set_xlabel('Loss')
ax[0].set_ylabel('Epochs')

ax[1].plot(history.history['accuracy'])
ax[1].set_title("accuracy after epochs")
ax[1].set_xlabel('accuracy')
ax[1].set_ylabel('Epochs')

plt.show()